In [1]:
import spacy
import pickle
import random

In [7]:
!pip install -U spacy

In [8]:
train_data = pickle.load(open('train_data.pkl', 'rb'))

In [9]:
train_data[0][1].get('entities')

[(1749, 1755, 'Companies worked at'),
 (1696, 1702, 'Companies worked at'),
 (1417, 1423, 'Companies worked at'),
 (1356, 1793, 'Skills'),
 (1209, 1215, 'Companies worked at'),
 (1136, 1248, 'Skills'),
 (928, 932, 'Graduation Year'),
 (858, 889, 'College Name'),
 (821, 856, 'Degree'),
 (787, 791, 'Graduation Year'),
 (744, 750, 'Companies worked at'),
 (722, 742, 'Designation'),
 (658, 664, 'Companies worked at'),
 (640, 656, 'Designation'),
 (574, 580, 'Companies worked at'),
 (555, 573, 'Designation'),
 (470, 493, 'Companies worked at'),
 (444, 469, 'Designation'),
 (308, 314, 'Companies worked at'),
 (234, 240, 'Companies worked at'),
 (175, 198, 'Companies worked at'),
 (93, 137, 'Email Address'),
 (39, 48, 'Location'),
 (13, 38, 'Designation'),
 (0, 12, 'Name')]

In [10]:
import warnings

In [15]:
nlp = spacy.blank('en')


def train_model(train_data):
    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe(str("ner"))
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")
        
    # add labels
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    
    #--------------------------
    
    # get names of other pipes to disable them during training
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
   
    # only train NER
    with nlp.disable_pipes(*other_pipes), warnings.catch_warnings():
        # show warnings for misaligned entity spans once
        warnings.filterwarnings("once", category=UserWarning, module='spacy')

        # reset and initialize the weights randomly – but only if we're
        # training a new model
        optimizer = nlp.begin_training()
            
        for itn in range(20):
            print('Starting iterations ', str(itn))
            random.shuffle(train_data)
            
            
            losses = {}
            index = 0
            for text, annotations in train_data:
                print(index)
                index = index + 1
                try:
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.2,  # dropout - make it harder to memorise data
                        sgd=optimizer,  # callable to update weights
                        losses=losses)
                except Exception as e:
                    pass
#                     print(text)
                
            print(losses)




In [16]:
train_model(train_data)

ValueError: [E966] `nlp.add_pipe` now takes the string name of the registered component factory, not a callable component. Expected string, but got <spacy.pipeline.ner.EntityRecognizer object at 0x000002BD2D8C19E0> (name: 'None').

- If you created your component with `nlp.create_pipe('name')`: remove nlp.create_pipe and call `nlp.add_pipe('name')` instead.

- If you passed in a component like `TextCategorizer()`: call `nlp.add_pipe` with the string name instead, e.g. `nlp.add_pipe('textcat')`.

- If you're using a custom component: Add the decorator `@Language.component` (for function components) or `@Language.factory` (for class components / factories) to your custom component and assign it a name, e.g. `@Language.component('your_name')`. You can then run `nlp.add_pipe('your_name')` to add it to the pipeline.